# Expectations related to our scenario

This notebook discusses expectations related to our organ transplant scenario. If you have not yet done so, please refer to **AgentsAndEvents.ipynb** for important events and fluents used in this scenario. For expectation examples, please refer to the **ExpectationExamples** folder.

There are many different categories of expectation that could be held by an agent in an product tracking scenario. Here, we consider the following categories: authorisation, compatibility, timely communication.

Each of these categories is covered by a separate notebook in the **ExpectationExamples** folder. Note that while the notebooks in the **ExpectationExample** folder should be able to function concurrently, these notebooks are run independently from one other to reduce the number of predicates loaded at once and simplify our query output.


### Set up

We need to set up our environment by loading in our event and fluent declarations from **dec:notation.pl** and **AE.pl**. The latter contains the code from **AgentsAndEvents.ipynb** the last time that notebook was run.

```diff
- IMPORTANT NOTE: you should run AgentsAndEvents.ipynb prior to running this notebook to ensure that AE.pl contains up to date predicates! You must run the notebook, not just update and save it.
```

In [1]:
?- cd('~/work'), ['dec:notation'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).
?- ['AE'].

true.
false.
true.
true.

### Expectation rule format

Expectation rules are declared for a particular agent. They are usually specified in an **initially/1** clause so that the expectation rule is active from time period zero onwards. 

We use an expectation rule with the form **exp_rule(Condition, Outcome, Status, Message))**. **Condition** refers to the triggering condition for the resulting expectation and **Outcome** is the state that needs to be reached for the expectation to be fulfilled. If **Status** is set to `independent`, then expectations resulting from our expectation will remain even if the expectation rule is deleted. **Message** is a more user friendly message describing the expectation rule, which uniquely identifies it, and which can also include a category for the expectation.

In our example expectation rule, we declare that the starting condition for our expectation is that **monSys** receives a wait list add request from a hospital, the resulting expected outcome is that the hospital is recognised by **monSys**, that the resulting expectations shall be dependent, and use the message "Authorisation":"Wait list requests should come from recognised hospital" to describe the expectation rule.

In [2]:
% File: authorisationExample.pl
% We make our initially/1 predicate dynamic to allow us to retract this expectation rule later
:- dynamic(initially/1).
initially(monSys:exp_rule(happ(receive(Hospital, waitAddReq(_, _, _))), agent(Hospital), dependent, 
"Authorisation":"Wait list requests should come from recognised hospital")).

### Expectation rule message categories

In order to test the example expectation rule added above, we add a narrative event in which Otago sends **monSys** a waiting list add request at time period 0.

In [3]:
% File: narrative.pl
happensAtNarrative("Otago":send(monSys, waitAddReq(101, ["liver", "kidney"], [sex:"F", 
dob:"01/01/2000"])),0).

At time period 0, the expectation rule holds for **monSys**. At time period 1, **monSys** receives Otago's waiting list add request, which fulfils our expectation that such requests will come from a recognised hospital.

In [4]:
?- run(10).
?- holdsAt(monSys, exp_rule(Condition, Outcome, Status, "Authorisation":MessageSpecifics), 0).
?- happensAt(monSys, Event, 1).

true.
Condition = happ(receive(_1770, waitAddReq(_1776, _1778, _1780))), Outcome = agent(_1770), Status = dependent, MessageSpecifics = b'Wait list requests should come from recognised hospital' .
Event = fulf(happ(receive(b'Otago', waitAddReq(101, [b'liver', b'kidney'], [Functor(188685,2,sex,b'F'), Functor(188685,2,dob,b'01/01/2000')]))), agent(b'Otago'), 1, agent(b'Otago'), dependent, :(b'Authorisation', b'Wait list requests should come from recognised hospital')) ;
Event = receive(b'Otago', waitAddReq(101, [b'liver', b'kidney'], [Functor(188685,2,sex,b'F'), Functor(188685,2,dob,b'01/01/2000')])) .

As our expectation rule messages can be of the form Category:Specifics, we are able to filter expectation rules by their category if we so desire:

In [5]:
?- run(10).
?- happensAt(monSys, fulf(_, _, _, Outcome, _, "Authorisation":MessageSpecifics), 1).

true.
Outcome = agent(b'Otago'), MessageSpecifics = b'Wait list requests should come from recognised hospital' .

# Expectation functionality

We reset our environment.

In [6]:
?- retractall(initially(_)).
?- cd('~/work'), ['dec:notation'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).
?- ['AE'].

true.
true.
true.
true.
true.

## Current outcome types

As well as having different categories of expectations in terms of their use for agents, we are able to have different types of expectations in terms of what classifies an expectation as fulfilled or violated.

We can create an expectation that given an event E :
- Another event will occur or a fluent will hold at the same time period
- An event will occur / fluent will hold in exactly X time periods after the current time period (`delay` clause)
- When an event occurs / fluent holds, another event will have occurred / fluent will have held exactly X time periods earlier (`preceded` clause)
- An event will occur / fluent will hold before X time periods are up (`within` clause)
- An event will not occur / fluent will not hold until X time periods are up (`later` clause)
- An event will eventually occur / a fluent will eventually hold (`eventually` clause)
- An event will always occur / a fluent will always hold from this time period onwards (`always` clause)
- Another event will occur / fluent will hold before a third event occurs / fluent holds (`before` clause)
- Another event will occur / fluent will hold until a third event occurs / fluent holds (`until` clause).
- **The opposite of any of these situations, and any logical combination of them.**

This allows us to create expectations such as the following:
- When a hospital sends a message to the monitoring agent, they will be a registered hospital known to the monitoring agent.
- A waiting list request will be either accepted or rejected within 1 time period.
- A transplant outcome notification will be be received by the monitoring agent within 6 time periods of the transplant being approved.
- A patient on the waiting list will eventually receive a transplant.

Many further expectation set ups could be supported by adding `eval` predicates to **dec:notation.pl**.

## Adjustable progress

The expectation outcomes listed above rely on being progressed at the end of each time period. Generally, expectations with a countdown time e.g. will occur within 3 time periods, are progressed by 1 unit each time period. (So if we have 4 time periods remaining in our expectation at time period 0, we only have 3 time periods remaining at time period 1, 2 time periods remaining at time period 2, and so on). However, we can also declare expectations which have adjustable countdown progression.



An expectation which seems particularly relevant in an organ donor situation is a `within` rule - which states that following an event, another state will hold **within** a certain period of time. Let us take the example of a rule stating that a waiting list request will be either accepted or rejected with a countdown value of 1. Note that this does not necessarily mean within 1 time periods of the request being received, but that when the request is received, the expectation will hold with a countdown value of 1, and we expect this value to drop over time. The expectation is fulfilled when the request is accepted or rejected before the countdown value drops below zero, and is violated if neither of these events occur before the countdown value drops below zero.

This is currently supported for `within` expectations only, but could be easily declared to apply for a wider range of expectations in **dec:notation.pl**.

### Declaring the levels

We use the total number of messages sent and received by **monSys** in the last 10 time periods as a measure of **monSys**' busyness, although other measures such as number of patients waiting for transplants or number of patients currently admitted at hospitals recognised by **monSys** could also be used. The `recentMessageTotal` fluent produces a count of the total number of messages sent and received by the current actor within the last 10 time periods. If less than 10 time periods have taken place since the start of the scenario, the total is taken from time zero to the current time period.

In [7]:
% File: recentMessageTotal.pl
holdsAt(Actor, recentMessageTotal(Total), CurrentTime):- 
    aggregate_all(count, ((happensAt(Actor, receive(Hosp, Message), T) ; 
    happensAt(Actor, send(Hosp, Message), T)), T < CurrentTime, T >= CurrentTime - 10), Total).

Level information is set using the fluent levels/3, where the first argument is a list of the level names, the second argument is the break points between the levels and the third argument is the weighting assigned to different types of events under the levels. 

In this example, we declare that there are three levels; low, medium and high, with break points at 4 and 6. The break points are set very low for ease of illustration, and mean that if there have been 4 or fewer messages sent and received within the last 10 time periods (inclusive of the current time period), the hospital's busyness is low, if there were between 4 (exclusive) and 6 (inclusive) messages, the hospital's busyness is medium, and if there were more than 6 messages, the hospital's busyness is high. At the current level settings, any match event is likely to lead to a high busyness level due to the large number of subsequent related events. Of course, in practice the break points would be high enough one match in isolation wasn't sufficient to achieve an increase in busyness level!

We also declare the relative weightings to be assigned to different types of **within** expectations. Here, `foreignAcceptance` expectations are given a weighting of 1 under all three levels, while `localAcceptance` expectations (representing actor decisions) are given a weighting of 1 under the low level, 0.5 under the medium level and 0.333 under the high level. `foreignAcceptance` expectations will be set to be those in which the current Actor expects that a offer or request they make will be accepted by another agent in a certain time frame. `localAcceptance` expectations will be set to expectations about acceptance made by the current agent. When a given hospital is busy, it may alter its expectations on how quickly it responds to requests, but will not necessarily alter its expectations on how quickly other agents respond to it. The `localAcceptance` and `foreignAcceptance` arguments can be specified as an additional parameter to a `within` clause.

In [8]:
% File: levels.pl
initially(monSys:levels([low,medium,high],[4,6],[foreignAcceptance=[1,1,1], localAcceptance=[1, 0.5, 0.333]])).

We declare our **monSys** expectation that a donation offer should be matched or a declaration of no match found will occur within 1 time period, and give the `within` clause a `localAcceptance` argument. We also declare another **monSys** expectation which states that a match offer should be accepted by the donor's hospital with a countdown value of three, and give its `within` clause a `foreignAcceptance` argument.

In [9]:
% File: adjustableExpectations.pl
% Donation offer to match expectation
initially(monSys:exp_rule(happ(receive(Hospital, donorOffer(ID, _, _))), 
within(or([happ(noMatch(Hospital, ID)), happ(match(_, Hospital, ID, _, _, _, _, _))]),1, localAcceptance), 
dependent, "Timely action":"A donation offer should be matched or a declaration of no match found will occur with a countdown value of one")).

initially(monSys:exp_rule(happ(send(DonorHospital, recipientFound(MatchID, _, _, _, _, _))),
within(happ(receive(DonorHospital, acceptMatch(MatchID))),3, foreignAcceptance), dependent, 
"Timely response":"A match offer should be accepted by the donor's hospital with a countdown value of three")).

The event weightings are used to affect how far to progress expectations under a given busyness level. If an expectation contains a *within* rule with the argument `foreignAcceptance`, and has a starting countdown value of 3, we expect the event to take place within the next three time periods, as `foreignAcceptance` `within` expectations (and any `within` expectations without a type specified) have a weighting of 1. Note that the starting countdown value is not necessarily the number of time periods before an expectation will be violated if the expected event does not take place. If an expectation with a *within* rule and `localAcceptance` argument instead stated that an a match event will take place and has a countdown time of 1, we would expect the event to take place within 3 time periods under a high busyness level, as 1 / 0.33 = 3. The weightings control how quickly the specified countdown valuedrops, and allow the rate of decrease in countdown value associated with some types of expectations to be modified by busyness level.

In [10]:
% File: levelRules.pl
% If the count is greater than the maximum breakpoint, the busyness level is the greatest level.
holdsAt(Actor, busyness(Level), T):- 
    holdsAt(Actor, recentMessageTotal(Count), T) , holdsAt(Actor, levels(Names,BreakPoints,_), T), 
    last(Names, Level), last(BreakPoints, P), Count > P.

% Else, check which breakpoints the count falls between, using 0 as the minimum breakpoint.
holdsAt(Actor, busyness(Level),T):-  
    holdsAt(Actor, recentMessageTotal(Count), T), holdsAt(Actor, levels(Names,BreakPoints,_), T), member(Level,Names), 
    nth0(LIndex,Names,Level), nth0(LIndex, BreakPoints, P),  P >= Count, 
    (LIndex > 0 -> (LIndex2 is LIndex - 1, nth0(LIndex2,BreakPoints,P2), P2 < Count); true).

% Decrease in countdown value should be set by weightings given in the levels/3 predicate.
progress(Actor, within(F1,T1,Type), within(F1,T2,Type),T):- 
    holdsAt(Actor, busyness(Level), T), holdsAt(Actor, levels(Levels,_,DecayList), T), member(Type=Decay,DecayList), 
    nth0(Index, Levels, Level), nth0(Index,Decay,D), T2 is T1 - D. 

We will use some narrative events to test out our busyness and adjustable expectation predicates. 

To simplify the narrative events listed, we will add have not included all `waitAddReq` messages referenced in `match` events. (To see an example of `waitAddReq` and `match` together, please refer to the [Timely Communication notebook](ExpectationExamples/Timely%20Communication.ipynb)).

- At time period 0, Wellington sends a `donorOffer` message to **monSys**.
- At time period 2, Otago sends a `waitAddReq` message to **monSys**.
- At time period 3, **monSys** creates a match.
- At time period 4, Christchurch sends a `donorOffer` message to **monSys**.
- At time period 7, **monSys** creates a match.

In [11]:
% File: narrative.pl
happensAtNarrative("Wellington":send(monSys, donorOffer(200, [heart, kidney], [bloodType:"A"])),0).
happensAtNarrative("Otago":send(monSys, waitAddReq(104, [kidney], [])), 2).
happensAtNarrative(monSys:match(701, "Wellington", 200, "Otago", 103, "Auckland", ["heart"], []),3).
happensAtNarrative("Christchurch":send(monSys, donorOffer(201, [kidney], [bloodType:"B"])),4).
%happensAtNarrative(monSys:match(702, "Christchurch", 201, "Otago", 104, "Auckland", ["kidney"], []),7).

We allow queries for the first 10 time periods.

In [12]:
?- run(9).

true.

At time period 1, **monSys** receives a `donorOffer` message from Wellington. This causes the expectation that the donor offer will be matched, or a no match event will occur, with a countdown value of one. No messages had been sent or received by **monSys** before time period one, so the busyness level is set to low.

In [13]:
?- happensAt(monSys, Event, 1).
?- holdsAt(monSys, exp(_, _, _, within(_,TimeRemaining,WithinType), _, Message), 1).
?- holdsAt(monSys, busyness(Level), 1).
?- holdsAt(monSys, recentMessageTotal(Total), 1).

Event = receive(b'Wellington', donorOffer(200, [Atom('688133'), Atom('688261')], [Functor(188685,2,bloodType,b'A')])) .
TimeRemaining = 1, WithinType = localAcceptance, Message = :(b'Timely action', b'A donation offer should be matched or a declaration of no match found will occur with a countdown value of one') .
Level = low .
Total = 0 .

As the busyness level was low at time period one, the `within` expectation countdown value dropped by one when it progressed and reached zero. Thus, a violation occurs, as no `match` or `noMatch` event occurs at time period 2 and the countdown value is less than or equal to zero.

In [14]:
?- T = 2, happensAt(monSys, viol(_, _, _, within(_,TimeRemaining,WithinType), _, Message), T).

T = 2, TimeRemaining = 0, WithinType = localAcceptance, Message = :(b'Timely action', b'A donation offer should be matched or a declaration of no match found will occur with a countdown value of one') .

At time period 3, a `match` event occurs for **monSys**. This causes a `recipientFound` message (amongst others) to be sent at time period 4. This creates the expectation that the match offer will be accepted by the recipient's hospital with a countdown value of 3. As the busyness level is still low, this countdown value will drop by 1 from 3 to 2 for time period 5.

In [15]:
?- happensAt(monSys, Event, 4).
?- holdsAt(monSys, exp(_, _, _, within(_,TimeRemaining,WithinType), _, Message), 4).
?- holdsAt(monSys, busyness(Level), 4).
?- holdsAt(monSys, recentMessageTotal(Total), 4).

Event = send(b'Wellington', recipientFound(701, 200, b'Otago', b'Auckland', [b'heart'], [])) ;
Event = send(b'Otago', donorFound(701, 103, b'Wellington', b'Auckland', [b'heart'], [])) ;
Event = send(b'Auckland', locationSelected(701, b'Wellington', b'Otago', [b'heart'], [])) ;
Event = send(b'Otago', waitAccept(104)) .
TimeRemaining = 3, WithinType = foreignAcceptance, Message = :(b'Timely response', b"A match offer should be accepted by the donor's hospital with a countdown value of three") .
Level = low .
Total = 2 .

At time period 5, **monSys** receives a `donorOffer` message from Christchurch. This creates the expectation that a `match` or `noMatch` event will occur by the next time period. The countdown value for the acceptance message from the recipient's hospital has dropped by 1 to 2 as expected.
The busyness level of **monSys** is now medium, as 6 messages have been sent or received by **monSys** in time periods 0-5.

In [16]:
?- happensAt(monSys, Event, 5).
?- holdsAt(monSys, exp(_, _, _, within(_,TimeRemaining,WithinType), _, Message), 5).
?- holdsAt(monSys, busyness(Level), 5).
?- holdsAt(monSys, recentMessageTotal(Total), 5).

Event = receive(b'Christchurch', donorOffer(201, [Atom('688261')], [Functor(188685,2,bloodType,b'B')])) .
TimeRemaining = 2, WithinType = foreignAcceptance, Message = :(b'Timely response', b"A match offer should be accepted by the donor's hospital with a countdown value of three") ;
TimeRemaining = 1, WithinType = localAcceptance, Message = :(b'Timely action', b'A donation offer should be matched or a declaration of no match found will occur with a countdown value of one') .
Level = medium .
Total = 6 .

As `foreignAcceptance` `within` expectations have a weighting of 1 under the medium busyness level, while `localAcceptance` `within` expectations have a weighting of 0.5 under the same level, the countdown value for our "Timely response" expectation decreases by one, while the countdown value for our "Timely action" expectation only decreases by 0.5 between time periods 5 and 6. At time period 6, the busyness level reaches high, with 7 messages being sent or received by **monSys** in the earlier time periods.

In [17]:
?- holdsAt(monSys, exp(_, _, _, within(_,TimeRemaining,WithinType), _, Message), 6).
?- holdsAt(monSys, busyness(Level), 6).
?- holdsAt(monSys, recentMessageTotal(Total), 6).

TimeRemaining = 1, WithinType = foreignAcceptance, Message = :(b'Timely response', b"A match offer should be accepted by the donor's hospital with a countdown value of three") ;
TimeRemaining = 0.5, WithinType = localAcceptance, Message = :(b'Timely action', b'A donation offer should be matched or a declaration of no match found will occur with a countdown value of one') .
Level = high .
Total = 7 .

At time period 7, the countdown value for our "Timely response" expectation has again decreased by one, while the countdown value for our "Timely action" expectation has decreased by 0.333 to 0.1667 (with some floating point error). As the countdown value for our "Timely response" expectation has reached zero and the expected events have not occurred, a violation occurs.

In [18]:
?- holdsAt(monSys, exp(_, _, _, within(_,TimeRemaining,WithinType), _, Message), 7).
?- happensAt(monSys, Event, 7).
?- holdsAt(monSys, busyness(Level), 7).


TimeRemaining = 0, WithinType = foreignAcceptance, Message = :(b'Timely response', b"A match offer should be accepted by the donor's hospital with a countdown value of three") ;
TimeRemaining = 0.16699999999999998, WithinType = localAcceptance, Message = :(b'Timely action', b'A donation offer should be matched or a declaration of no match found will occur with a countdown value of one') .
Event = viol(happ(send(b'Wellington', recipientFound(701, 200, b'Otago', b'Auckland', [b'heart'], []))), within(happ(receive(b'Wellington', acceptMatch(701))), 3, foreignAcceptance), 4, within(happ(receive(b'Wellington', acceptMatch(701))), 0, foreignAcceptance), dependent, :(b'Timely response', b"A match offer should be accepted by the donor's hospital with a countdown value of three")) .
Level = high .

At time period 8, the countdown value for our "Timely response" expectation has dropped below zero, so a violation (finally!) occurs. Note that this expectation was created at time period 5 with a countdown value of 1. Under a low busyness level, a violation would have occurred at time period 6. However, due to the increased busyness levels, the drop in countdown value was diminished and it took an additional 2 time periods for a violation to occur.

In [19]:
?- holdsAt(monSys, exp(_, _, _, within(_,TimeRemaining,WithinType), _, Message), 8).
?- happensAt(monSys, Event, 8).
?- holdsAt(monSys, busyness(Level), 8).


TimeRemaining = -0.16600000000000004, WithinType = localAcceptance, Message = :(b'Timely action', b'A donation offer should be matched or a declaration of no match found will occur with a countdown value of one') .
Event = viol(happ(receive(b'Christchurch', donorOffer(201, [Atom('688261')], [Functor(188685,2,bloodType,b'B')]))), within(or([Functor(14508173,1,noMatch(b'Christchurch', 201)), Functor(14508173,1,match(_1734, b'Christchurch', 201, _1740, _1742, _1744, _1746, _1748))]), 1, localAcceptance), 5, within(or([Functor(14508173,1,noMatch(b'Christchurch', 201)), Functor(14508173,1,match(_1734, b'Christchurch', 201, _1740, _1742, _1744, _1746, _1748))]), -0.16600000000000004, localAcceptance), dependent, :(b'Timely action', b'A donation offer should be matched or a declaration of no match found will occur with a countdown value of one')) .
Level = high .

As illustrated above, we are able to have an event trigger an expectation rule at one busyness level, and  the drop in the associated countdown value will be affected by the busyness level during future time periods, until the expectation is either fulfilled or violated. This allows us to extend the time period specified for **within** expectations under busy scenarios.

## Independent expectations

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=32f94018-a4da-40ef-8c9f-8983d73811c8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>